In [12]:
from neo4j import GraphDatabase

In [13]:
class Neo4jConnection:    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [14]:
conn = Neo4jConnection(uri="bolt://localhost:7687", 
                       user="neo4j",              
                       pwd="abcd")

##### Creating Products graph nodes

In [15]:
query1 = '''LOAD CSV WITH HEADERS FROM "https://data.neo4j.com/northwind/products.csv" AS row
CREATE (n:Product)
SET n = row,
n.unitPrice = toFloat(row.unitPrice),
n.unitsInStock = toInteger(row.unitsInStock), n.unitsOnOrder = toInteger(row.unitsOnOrder),
n.reorderLevel = toInteger(row.reorderLevel), n.discontinued = (row.discontinued <> "0")
'''

In [16]:
conn.query(query1)

[]

##### Creating Categories graph nodes

In [18]:
q2 = '''LOAD CSV WITH HEADERS FROM "https://data.neo4j.com/northwind/categories.csv" AS row
CREATE (n:Category)
SET n = row'''

In [19]:
conn.query(q2)

[]

##### Creating Suppliers graph nodes

In [20]:
q3 = '''LOAD CSV WITH HEADERS FROM "https://data.neo4j.com/northwind/suppliers.csv" AS row
CREATE (n:Supplier)
SET n = row'''

In [21]:
conn.query(q3)

[]

##### Creating Indices

In [22]:
q4 = '''CREATE INDEX FOR (p:Product) ON (p.productID)'''
q5 = '''CREATE INDEX FOR (p:Product) ON (p.productName)'''
q6 = '''CREATE INDEX FOR (c:Category) ON (c.categoryID)'''

In [23]:
conn.query(q4)
conn.query(q5)
conn.query(q6)

[]

In [24]:
q7 = '''MATCH (p:Product),(c:Category)
WHERE p.categoryID = c.categoryID
CREATE (p)-[:PART_OF]->(c)'''

In [25]:
conn.query(q7)

[]

In [26]:
q8 = '''MATCH (p:Product),(s:Supplier)
WHERE p.supplierID = s.supplierID
CREATE (s)-[:SUPPLIES]->(p)'''

In [27]:
conn.query(q8)

[]

In [28]:
q9 = '''MATCH (s:Supplier)-->(:Product)-->(c:Category)
RETURN s.companyName as Company, collect(distinct c.categoryName) as Categories'''

In [29]:
conn.query(q9)

[<Record Company='Bigfoot Breweries' Categories=['Beverages']>,
 <Record Company='Pavlova' Categories=['Beverages', 'Condiments', 'Confections', 'Meat/Poultry', 'Seafood']>,
 <Record Company='Aux joyeux ecclésiastiques' Categories=['Beverages']>,
 <Record Company='Karkki Oy' Categories=['Beverages', 'Confections']>,
 <Record Company='Leka Trading' Categories=['Beverages', 'Condiments', 'Grains/Cereals']>,
 <Record Company='Plutzer Lebensmittelgroßmärkte AG' Categories=['Beverages', 'Condiments', 'Grains/Cereals', 'Meat/Poultry', 'Produce']>,
 <Record Company='Refrescos Americanas LTDA' Categories=['Beverages']>,
 <Record Company='Exotic Liquids' Categories=['Beverages', 'Condiments']>,
 <Record Company="Forêts d'érables" Categories=['Condiments', 'Confections']>,
 <Record Company='New Orleans Cajun Delights' Categories=['Condiments']>,
 <Record Company="Mayumi's" Categories=['Condiments', 'Produce', 'Seafood']>,
 <Record Company="Grandma Kelly's Homestead" Categories=['Condiments', 'Pr

In [30]:
q10 = '''MATCH (c:Category {categoryName:"Produce"})<--(:Product)<--(s:Supplier)
RETURN DISTINCT s.companyName as ProduceSuppliers'''

In [31]:
conn.query(q10)

[<Record ProduceSuppliers="G'day">,
 <Record ProduceSuppliers='Tokyo Traders'>,
 <Record ProduceSuppliers='Plutzer Lebensmittelgroßmärkte AG'>,
 <Record ProduceSuppliers="Mayumi's">,
 <Record ProduceSuppliers="Grandma Kelly's Homestead">]

In [32]:
q11 = '''LOAD CSV WITH HEADERS FROM "https://data.neo4j.com/northwind/customers.csv" AS row
CREATE (n:Customer)
SET n = row'''

In [33]:
conn.query(q11)

[]

In [34]:
q12 = '''LOAD CSV WITH HEADERS FROM "https://data.neo4j.com/northwind/orders.csv" AS row
CREATE (n:Order)
SET n = row'''

In [35]:
conn.query(q12)

[]

In [37]:
q13 = '''CREATE INDEX FOR (n:Customer) ON (n.customerID)'''
q14 = '''CREATE INDEX FOR (o:Order) ON (o.orderID)'''

In [38]:
conn.query(q13)
conn.query(q14)

[]

In [39]:
q15 = '''MATCH (n:Customer),(o:Order)
WHERE n.customerID = o.customerID
CREATE (n)-[:PURCHASED]->(o)
'''

In [40]:
conn.query(q15)

[]

In [41]:
q16 = '''LOAD CSV WITH HEADERS FROM "https://data.neo4j.com/northwind/order-details.csv" AS row
MATCH (p:Product), (o:Order)
WHERE p.productID = row.productID AND o.orderID = row.orderID
CREATE (o)-[details:ORDERS]->(p)
SET details = row,
details.quantity = toInteger(row.quantity)'''

In [42]:
conn.query(q16)

[]

In [43]:
q17 = '''MATCH (cust:Customer)-[:PURCHASED]->(:Order)-[o:ORDERS]->(p:Product),
  (p)-[:PART_OF]->(c:Category {categoryName:"Produce"})
RETURN DISTINCT cust.contactName as CustomerName, SUM(o.quantity) AS TotalProductsPurchased'''

In [44]:
conn.query(q17)

[<Record CustomerName='Maria Larsson' TotalProductsPurchased=148>,
 <Record CustomerName='Hanna Moos' TotalProductsPurchased=11>,
 <Record CustomerName='Mario Pontes' TotalProductsPurchased=35>,
 <Record CustomerName='Isabel de Castro' TotalProductsPurchased=18>,
 <Record CustomerName='Carine Schmitt' TotalProductsPurchased=3>,
 <Record CustomerName='Patricia McKenna' TotalProductsPurchased=121>,
 <Record CustomerName='Bernardo Batista' TotalProductsPurchased=17>,
 <Record CustomerName='Sergio Gutiérrez' TotalProductsPurchased=7>,
 <Record CustomerName='Horst Kloss' TotalProductsPurchased=242>,
 <Record CustomerName='Hari Kumar' TotalProductsPurchased=72>,
 <Record CustomerName='Jytte Petersen' TotalProductsPurchased=30>,
 <Record CustomerName='Zbyszek Piestrzeniewicz' TotalProductsPurchased=22>,
 <Record CustomerName='Yang Wang' TotalProductsPurchased=71>,
 <Record CustomerName='Carlos Hernández' TotalProductsPurchased=81>,
 <Record CustomerName='Thomas Hardy' TotalProductsPurchased=2

In [45]:
q18 = '''MATCH (cust:Customer)-[:PURCHASED]->(:Order)-[o:ORDERS]->(p:Product),
  (p)-[:PART_OF]->(c:Category {categoryName:"Produce"})
RETURN DISTINCT cust.contactName as CustomerName, SUM(o.quantity) AS TotalProductsPurchased
ORDER BY TotalProductsPurchased DESC'''

In [49]:
conn.query(q18)[0]['CustomerName']

'Roland Mendel'